# Slivka Client Demo

## Introduction

This notebook demonstrates how to use the Slivka Python client to interact with a Slivka server. You'll learn how to install the client, query available services, submit a job, and retrieve the results.

### Install Slivka Client

In [1]:
%pip install requests

import requests
import zipfile
import io

# Download the repository as a zip file
url = "https://github.com/bartongroup/slivka-python-client/archive/refs/heads/dev.zip"
response = requests.get(url)
if response.status_code == 200:
	zip_file = zipfile.ZipFile(io.BytesIO(response.content))
	zip_file.extractall("slivka-python-client")
	%pip install ./slivka-python-client/slivka-python-client-dev
else:
	print("Failed to download the zip file. Status code:", response.status_code)


Note: you may need to restart the kernel to use updated packages.
Processing ./slivka-python-client/slivka-python-client-dev
  Preparing metadata (setup.py) ... done
  Created wheel for slivka-client: filename=slivka_client-1.2.1b1-py3-none-any.whl size=13422 sha256=7dabeb3381cc7f55603abe8c35a860b40ea2804f7c6a72fafa71affd49664365
  Stored in directory: /home/jovyan/.cache/pip/wheels/e5/d8/11/60dd3f9ca2b38d63c070cb2fd43d9ae68892a615e0dc32375d
Successfully built slivka-client
  Attempting uninstall: slivka-client
    Found existing installation: slivka-client 1.2.1b1
    Uninstalling slivka-client-1.2.1b1:
      Successfully uninstalled slivka-client-1.2.1b1
Note: you may need to restart the kernel to use updated packages.


### Configure Slivka Server URL

In [2]:
# Set the Slivka server URL
local_slivka_url = 'http://slivka-bio:8000/'  # For Slivka running in Docker
dundee_slivka_url = 'https://www.compbio.dundee.ac.uk/slivka/'

# Set to True to use the local Slivka server
use_local_slivka = True

slivka_server_url = local_slivka_url if use_local_slivka else dundee_slivka_url
print(f"Using Slivka server at: {slivka_server_url}")


Using Slivka server at: http://slivka-bio:8000/


### Initialize Slivka Client

In [3]:
from slivka_client import SlivkaClient

client = SlivkaClient(slivka_server_url)
print(f"Connected to Slivka server at: {client.url}")


Connected to Slivka server at: http://slivka-bio:8000/


### Query Available Services

In [4]:
# List available services
for service in client.services:
    print(f"Service name: {service.name}, ID: {service.id}")


Service name: MAFFT, ID: mafft
Service name: ProbCons, ID: probcons
Service name: MSAProbs, ID: msaprobs
Service name: ClustalO, ID: clustalo
Service name: GlobPlot, ID: globplot
Service name: MUSCLE, ID: muscle
Service name: ClustalW, ID: clustalw
Service name: DisEMBL, ID: disembl
Service name: JPred, ID: jpred
Service name: Jronn, ID: jronn
Service name: TCoffee, ID: tcoffee
Service name: IUPred, ID: iupred
Service name: AACon, ID: aacon
Service name: RNAalifold, ID: rnaalifold
Service name: Example Service, ID: example


### Select a Service (ClustalO)


In [5]:
service = client['clustalo']
print(f"Selected service: {service.name} (ID: {service.id})")


Selected service: ClustalO (ID: clustalo)


### Submit a Job to Align Sequences with ClustalO


In [6]:
# Define parameters for the job
data = {
    'iterations': 1,
    'max-guidetree-iterations': 1,
    'max-hmm-iterations': 1,
}

# Specify the input file path
input_file_path = 'data/fam83b-pandda-chains-for-align-test.fa'

# Submit the job with the input file
with open(input_file_path, 'rb') as file_object:
    files = {'input': file_object}
    job = service.submit_job(data=data, files=files)
    print(f"Job submitted with ID: {job.id}")


Job submitted with ID: ZxjSA5S059PP66p1


### Monitor Job Status


In [7]:
import time

# Poll the job status until it is completed
while job.status not in ('COMPLETED', 'FAILED'):
    print(f"Job status: {job.status}")
    time.sleep(3)

print(f"Job completed with status: {job.status}")


Job status: PENDING
Job status: PENDING
Job completed with status: COMPLETED


### Retrieve Job Results


In [8]:
import os

# Download each file in the job results
for file in job.files:
    download_path = f"./downloads/{file.id}"
    os.makedirs(os.path.dirname(download_path), exist_ok=True)
    file.dump(download_path)
    print(f"File {file.id} downloaded to {download_path}")

File ZxjSA5S059PP66p1/output.txt downloaded to ./downloads/ZxjSA5S059PP66p1/output.txt
File ZxjSA5S059PP66p1/stat.log downloaded to ./downloads/ZxjSA5S059PP66p1/stat.log
File ZxjSA5S059PP66p1/stderr downloaded to ./downloads/ZxjSA5S059PP66p1/stderr
